In [5]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data from csv file
df = pd.read_csv("student_data.csv")

#Extract the number of students from the dataframe
num_students = len(df)


#Create a list of students from 0 to num_students-1 for indexing purposes
students = list(range(num_students))

#craete a list of teams from 0 to num_teams-1 for indexing purposes
NUM_TEAMS = len(students)//2

#create a dictionary of student preferences
#The key is the student number and the value is a list of the student's preferences
#The number represents the other student 
#For example, if preference 1 is 15 then the student prefers student 15 over all other students
#If the student has no preference for a particular student, then the preference is -1
preferences = {}
for i in students:
    preferences[i] = df.iloc[i,1:6].tolist()
    

#optimization model
m = gp.Model("student")

#Decision variables
#Decision variable that indicates if student i is assigned to team j
x = m.addVars(students, range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="x")

#create an auxiliary variable for each team that indicates the number of students in that team
y = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.INTEGER, name="y")

#create a preference score for each team that is the sum of the preference scores of all students in that team
#Calculate the average preference score for each team
pref = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.INTEGER, name="pref")


# Define objective function
#The objective is to maximize the preference score of all teams, and minimize the number of teams 
m.setObjective(pref.sum(), GRB.MAXIMIZE)


#Add constraints
#Each student must be assigned to exactly one team
m.addConstrs((x.sum(i,'*') == 1 for i in students), name="student_team")

#Eah team must have least 2 students but at most 3 students
m.addConstrs((x.sum('*',j) >= 2 for j in range(NUM_TEAMS)), name="min_team_size")
m.addConstrs((x.sum('*',j) <= 3 for j in range(NUM_TEAMS)), name="max_team_size")

#add a constraint that has a max preference score of 10 for each team
m.addConstrs((pref[j] <= 10 for j in range(NUM_TEAMS)), name="max_pref_score")


#optimize model
m.optimize()

#print the teams and what students are in each team as a table
print("Teams")
print("Team Number, Student 1, Student 2, Student 3")
for j in range(NUM_TEAMS):
    print(j, end=", ")
    for i in students:
        if x[i,j].x == 1:
            print(i, end=", ")
    print()
    



Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 46 rows, 189 columns and 522 nonzeros
Model fingerprint: 0x0bcb018f
Variable types: 0 continuous, 189 integer (171 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 90.0000000
Presolve removed 9 rows and 18 columns
Presolve time: 0.00s
Presolved: 37 rows, 171 columns, 513 nonzeros
Variable types: 0 continuous, 171 integer (171 binary)

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 90 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e+01, best bound 9.000000000000e+

The code below assigns weights if a student is assigned to one of their preferences. +10 if assigned to 1st preference, +5 if assigned to 2nd preference, and +2 if assigned to 3rd preference

Objective function: Goal is to maximize preference scores of all teams, and minimize the number of teams created 


In [5]:
%pip install -q gurobipy
%pip install -q pandas
%pip install -q numpy
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Load data from csv file
preferences_dict = {}

for i in range(1,21):
    preferences_dict[i] = [np.random.randint(1, 21) for i  in range(3)]

print(preferences_dict)

df = pd.read_csv("student_data1.csv")

#Extract the number of students from the dataframe
num_students = len(preferences_dict)

#Create a list of students from 0 to num_students-1 for indexing purposes
students = list(range(num_students))

#compute number of teams from 0 to num_teams-1 for indexing purposes
NUM_TEAMS = len(students)//2

#create a dictionary of student preferences
#The key is the student inedx, and the value is a list of the student's preferences
#Each integer in the preferences list represents a unique student index 
#For example, if the first value in the list is 15 then the student prefers student 15 over all other students
#If the student has no preference for a particular student, then the preference is -1
preferences = {}
for i in students:
    preferences[i] = df.iloc[i,1:6].tolist()

#optimization model
m = gp.Model("student")

#Decision variables
#Decision variable that indicates if student i is assigned to team j
x = m.addVars(students, range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="x")


#create an auxiliary variable for each team that indicates the number of students in that team
y = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.INTEGER, name="y")

#decision variable that indicates if student i is assigned to their first choice
z = m.addVars(students, range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="z")

#decision variable that indicates if student i is assigned to their second choice
z2 = m.addVars(students, range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="z2")

#decision variable that indicates if student i is assigned to their third choice
z3 = m.addVars(students, range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="z3")


#create a preference score for each team that is the sum of the preferences of the students in the team for their assigned partner
#use the decision variables z, z2, and z3 to calculate the preference score
#Calculate the average preference score for each team
#assign weights to the preferences so that the first choice is weighted more than the second choice and the second choice is weighted more than the third choice

pref = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.INTEGER, name="pref")
for j in range(NUM_TEAMS):
    for i in students:
        if preferences[i][0] != -1:
            pref[j] += z[i,j]*preferences[i][0]*10
        if preferences[i][1] != -1:
            pref[j] += z2[i,j]*preferences[i][1]*5
        if preferences[i][2] != -1:
            pref[j] += z3[i,j]*preferences[i][2]*2


# Define objective function
#The objective is to maximize the average preference score of each team, minimize the number of teams, and penalize the number of students that are not assigned to their first choice
m.setObjective(pref.sum() - 1000*(num_students - z.sum()) - 1000*(num_students - z2.sum()) - 1000*(num_students - z3.sum()), GRB.MAXIMIZE)


#Add constraints
#Each student must be assigned to exactly one team
m.addConstrs((x.sum(i,'*') == 1 for i in students), name="student_team")

#Eah team must have least 2 students but at most 3 students
m.addConstrs((x.sum('*',j) >= 2 for j in range(NUM_TEAMS)), name="min_team_size")
m.addConstrs((x.sum('*',j) <= 3 for j in range(NUM_TEAMS)), name="max_team_size")

#Bound the preference score for each team to the objective function
m.addConstrs((pref[j] <= pref.sum() for j in range(NUM_TEAMS)), name="max_pref_score")


#add a constraint that each team has a student assigned to their first choice
m.addConstrs((z.sum('*',j) >= 1 for j in range(NUM_TEAMS)), name="min_first_choice")



#optimize model
m.optimize()

 # Print a table with team number, team members, and the average preference score for each team
print("Teams")
print("Team Number, Student 1, Student 2, Student 3, Average Preference Score")
for j in range(NUM_TEAMS):
    print(j, end=", ")
    for i in students:
        if x[i,j].x == 1:
            print(i, end=", ")
    print(pref[j].getValue())





[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
{1: [19, 7, 7], 2: [5, 20, 8], 3: [15, 4, 15], 4: [11, 5, 1], 5: [4, 14, 1], 6: [15, 15, 4], 7: [12, 9, 5], 8: [3, 14, 6], 9: [3, 1, 20], 10: [11, 7, 2], 11: [10, 11, 11], 12: [8, 14, 1], 13: [17, 16, 5], 14: [7, 10, 11], 15: [2, 9, 19], 16: [18, 11, 6], 17: [9, 10, 18], 18: [6, 3, 13], 19: [13, 4, 4]}
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical

GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license